In [1]:
import pandas as pd
import s3fs
import numpy as np
import swifter

In [2]:
def f(lang, embeddings, index):
    if index % 1000000  == 0:
        print(index)
    return [f'{lookup[lang]+i}:{e}' for i, e in enumerate(embeddings)]

In [3]:
new_names = list(range(1,21))
new_names = list(map(str, new_names))

In [ ]:
all_features = ["language"]
train = pd.read_csv("s3://recsys-challenge-2020/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [7], sep="\x01"
                   )
print('train language loaded')

In [ ]:
language_code_name_pairs = pd.read_csv('s3://recsys-challenge-2020/language_code_name_pairs.csv')

In [ ]:
train_input = pd.merge(train, language_code_name_pairs, how = 'inner',\
left_on = 'language', right_on = 'code')[['name']]

In [ ]:
all_lang_output_train = pd.read_csv('s3://recsys-challenge-2020/all_lang_output_train.csv')
lang_embedding_train = pd.concat([all_lang_output_train[['sentence_embedding']], train_input[['name']]], axis = 1)
lang_embedding_train.sentence_embedding = lang_embedding_train.sentence_embedding.apply(lambda x: np.fromstring(x[1:-1], sep=' '))
lang_embedding_train = lang_embedding_train.rename(columns={"name": "language"})

In [ ]:
start_feature_number = 42963998

In [ ]:
lookup = {l:start_feature_number+i*20 for i, l in enumerate(lang_embedding_train.language.drop_duplicates().to_list())}

In [ ]:
lookup

In [ ]:
for chunk in np.array_split(lang_embedding_train,20):
    chunk_embeddings_list = pd.DataFrame()
    chunk_embeddings_list = chunk.swifter.apply(lambda x: f(x.language, x.sentence_embedding, x.name), axis=1, result_type='expand')
    chunk_embeddings_list.to_csv('data/lang_embedd_train.csv', header = False, index = False, mode = 'a', sep = ' ')